In [1]:
import xarray as xr
import pandas as pd
import matplotlib.pylab as plt
import cartopy.crs as ccrs
import numpy as np
import matplotlib.cm as cm
import cmocean.cm as cmo
import intake
import numcodecs
import gribscan
from scipy.interpolate import CloughTocher2DInterpolator, LinearNDInterpolator, NearestNDInterpolator
import scipy.spatial

In [2]:
from dask.distributed import Client
import dask
dask.config.config.get('distributed').get('dashboard').update({'link':'{JUPYTERHUB_SERVICE_PREFIX}/proxy/{port}/status'})

client = Client(n_workers=15, threads_per_worker=1, memory_limit='12GB')
client.run(numcodecs.register_codec, gribscan.RawGribCodec, "gribscan.rawgrib")


{'tcp://127.0.0.1:34243': None,
 'tcp://127.0.0.1:34783': None,
 'tcp://127.0.0.1:35411': None,
 'tcp://127.0.0.1:35819': None,
 'tcp://127.0.0.1:36703': None,
 'tcp://127.0.0.1:36945': None,
 'tcp://127.0.0.1:37041': None,
 'tcp://127.0.0.1:37563': None,
 'tcp://127.0.0.1:38713': None,
 'tcp://127.0.0.1:38769': None,
 'tcp://127.0.0.1:39727': None,
 'tcp://127.0.0.1:41027': None,
 'tcp://127.0.0.1:44583': None,
 'tcp://127.0.0.1:44845': None,
 'tcp://127.0.0.1:45827': None}

In [3]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/a270088/levante-spawner-preset//proxy/8787/status,
Dashboard: /user/a270088/levante-spawner-preset//proxy/8787/status,Workers: 15
Total threads: 15,Total memory: 167.64 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:41573,Workers: 15
Dashboard: /user/a270088/levante-spawner-preset//proxy/8787/status,Total threads: 15
Started: Just now,Total memory: 167.64 GiB
Comm: tcp://127.0.0.1:37041,Total threads: 1
Dashboard: /user/a270088/levante-spawner-preset//proxy/34993/status,Memory: 11.18 GiB
Nanny: tcp://127.0.0.1:45917,


In [4]:
cat = intake.open_catalog("../catalog.yaml")

In [5]:
run = cat.IFS["tco2559-ng5"]

In [6]:
data = run.ICMGG_atm2d.to_dask()

In [7]:
daily = data['sst'].resample(time='1D').mean(dim='time')

In [8]:
daily

<xarray.DataArray 'sst' (time: 348, value: 26306560)>
dask.array<stack, shape=(348, 26306560), dtype=float64, chunksize=(1, 26306560), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2020-01-20 2020-01-21 ... 2021-01-01
    lat      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
    lon      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
Dimensions without coordinates: value

In [10]:
for ttime in range(daily.time.shape[0]):
    variable_name='sst'
    strtime=strtime = daily[ttime].time.values.astype('str')[:16]
    ostrtime = strtime.replace(':','_')
    daily[ttime,:].to_netcdf(f'/work/ab0995/a270088/NextGems_public/reductions/tco2559-ng5/sst/daily/{variable_name}_{ostrtime}_{str(ttime).zfill(10)}.nc')
    print(ostrtime)

2020-01-20T00_00
2020-01-21T00_00
2020-01-22T00_00
2020-01-23T00_00
2020-01-24T00_00
2020-01-25T00_00
2020-01-26T00_00
2020-01-27T00_00
2020-01-28T00_00
2020-01-29T00_00
2020-01-30T00_00
2020-01-31T00_00
2020-02-01T00_00
2020-02-02T00_00
2020-02-03T00_00
2020-02-04T00_00
2020-02-05T00_00
2020-02-06T00_00
2020-02-07T00_00
2020-02-08T00_00
2020-02-09T00_00
2020-02-10T00_00
2020-02-11T00_00
2020-02-12T00_00
2020-02-13T00_00
2020-02-14T00_00
2020-02-15T00_00
2020-02-16T00_00
2020-02-17T00_00
2020-02-18T00_00
2020-02-19T00_00
2020-02-20T00_00
2020-02-21T00_00
2020-02-22T00_00
2020-02-23T00_00
2020-02-24T00_00
2020-02-25T00_00
2020-02-26T00_00
2020-02-27T00_00
2020-02-28T00_00
2020-02-29T00_00
2020-03-01T00_00
2020-03-02T00_00
2020-03-03T00_00
2020-03-04T00_00
2020-03-05T00_00
2020-03-06T00_00
2020-03-07T00_00
2020-03-08T00_00
2020-03-09T00_00
2020-03-10T00_00
2020-03-11T00_00
2020-03-12T00_00
2020-03-13T00_00
2020-03-14T00_00
2020-03-15T00_00
2020-03-16T00_00
2020-03-17T00_00
2020-03-18T00_

In [4]:
data = xr.open_mfdataset('/work/ab0995/a270088/NextGems_public/reductions/tco2559-ng5/sst/daily/*.nc', 
                         decode_cf=False,
                         combine='nested',
                         concat_dim ='time', 
                         chunks={'time':1}
                        )

In [5]:
data

<xarray.Dataset>
Dimensions:  (time: 348, value: 26306560)
Coordinates:
  * time     (time) int64 0 0 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0 0
Dimensions without coordinates: value
Data variables:
    lat      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    lon      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    sst      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>

In [6]:
time = pd.date_range('2020-01-20 00:00', '2021-01-01 00:00', freq='D')

In [7]:
time[-1]

Timestamp('2021-01-01 00:00:00', freq='D')

In [8]:
data = data.assign_coords({'time':time})

In [9]:
data

<xarray.Dataset>
Dimensions:  (time: 348, value: 26306560)
Coordinates:
  * time     (time) datetime64[ns] 2020-01-20 2020-01-21 ... 2021-01-01
Dimensions without coordinates: value
Data variables:
    lat      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    lon      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    sst      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>

In [10]:
data.to_netcdf('/work/ab0995/a270088/NextGems_public/reductions/tco2559-ng5/tco2559-ng5.sst.daily.nc', 
              encoding={
            "time": {"dtype": np.dtype("double")},
            "lat": {"dtype": np.dtype("double")},
            "lon": {"dtype": np.dtype("double")},
            'sst': {"dtype": np.dtype("single")},
        },)